For reading the dataset 

In [14]:
import pandas as pd
import numpy as np
from copy import deepcopy
from distutils.command.build_scripts import first_line_re
from tkinter.tix import COLUMN
# Import deque for the stack structure, copy for deep copy nodes
from collections import deque


class DatasetPart2:
    def __init__(self,df):
        self.df=df
        self.x = self.df.iloc[:,:-1]
        self.y = self.df.iloc[:,-1]
        # self.y = self.df.iloc[:-1]

    @classmethod
    def constructFromFile(cls,filePath):
        df = pd.read_csv(filePath,header=None)
        df.columns = [f"feature{i+1}" for i in range(len(df.columns))]
        df.rename(columns = {f'feature{len(df.columns)}':'class'}, inplace = True)
        return cls(df) 

class Sonar(DatasetPart2):
    def __init__(self,df):
        super().__init__(df)
    
class Wbcd(DatasetPart2):
    def __init__(self,df):
        super().__init__(df)

ds_sonar  = Sonar.constructFromFile("./sonar/sonar.data")
ds_wbcd = Wbcd.constructFromFile("./wbcd/wbcd.data")
ds_wbcd.df
ds_sonar.df


,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature52,feature53,feature54,feature55,feature56,feature57,feature58,feature59,feature60,class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,1
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,1
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,1
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,1
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,2
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,2
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,2
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,2


In [8]:

# define some constants for the genetic algorithm
CONSTANTS_DICT = {
    "POPULATION_SIZE": 100, # number of individuals in each population
    "MAX_GENERATIONS": 250, # number of generations to run the algorithm
    "CROSSOVER_RATE": 1.0, # crossover rate should always be 100%, based on slides
    "MUTATION_RATE": 0.2, # mutation rate
    "ELITIST_PERCENTAGE": 0.05, # percentage of the best individuals to keep in the next generation
    
}


> Use **Creator** to define the type of individuals and fitness classes.

In [9]:
from asyncio import constants
from json import tool
from deap import creator, base, gp, tools, algorithms # core functionality of DEAP
import array
import random
import json
import math # for checking the fitness of an individual, i.e. math.isinf(weight)
import matplotlib.pyplot as plt
# Python internal operators for object comparisons, 
# logical comparisons, arithmetic operations, and sequence operations
import operator 

# creator is  usually used to define the type of the individual and fitness classes

# goal:to maximize the value and do not exceed the capacity of the knapsack
# define strategies with different priorities for optimizing multiple goals by using FitnessCompound
# 1 for maximize value, -1 for minimize weight, 
# creator.create("FitnessCompound", base.Fitness, weights=(1.0,-1.0)) 
 
# according to slide, fitness value has been reduced to 1 dimension, so just use FitnessMax
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# Individual should be a list of binary values, i.e. a list of 0s and 1s
creator.create("Individual", list, fitness=creator.FitnessMax)


/usr/pkg/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/pkg/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


> Define the evaluate function for **FilterGA** and **WrapperGA**.
>
> Inspired by the slide.

In [10]:
def evaluateFilterGA(ds:DatasetPart2, individual:creator.Individual): 
    
    pass

def evaluateWrapperGA(ds:DatasetPart2, individual:creator.Individual): 
    pass


Setup toolbox for registering the functions

In [15]:

# toolbox is a class contains the operators that we will use in our genetic programming algorithm
# it can be also be used as the container of methods which enables us to add new methods to the toolbox 
def setup_toolbox(ds:DatasetPart2, evaluateFunction,randSeed:int) -> base.Toolbox:
    toolbox = base.Toolbox()
    # for population size, we use the random.randint function to generate a random integer in the range [min, max]
    random.seed(randSeed)
    # register a method to generate random boolean values
    toolbox.register("attr_bool", random.randint, 0, 1)
    # register a method to generate random individuals
    toolbox.register("IndividualCreator", 
                     tools.initRepeat, 
                     creator.Individual, 
                     toolbox.attr_bool, 
                     n=len(ds.x.columns) # feature number, exclude the class column
                    )
    
    # N is not specificied, so need to specify number of individuals to generate within each population when we call it later
    toolbox.register("PopulationCreator", tools.initRepeat, list, toolbox.IndividualCreator) 
    
    toolbox.register("elitism", tools.selBest, k=int(CONSTANTS_DICT["ELITIST_PERCENTAGE"]*ds.df.shape[0]))
    toolbox.register("select", tools.selTournament, k=2, tournsize=3)
    
    toolbox.register("mate", tools.cxTwoPoint) # TODO: might need to change this to cxOnePoint
    # indpb refer to the probability of mutate happening on each gene, it is NOT the same as mutation rate
    toolbox.register("mutate", tools.mutFlipBit, indpb=1.0/ds.M) # TODO: might need to change this to mutUniformInt
    # local search operator
    # toolbox.register("local_search", algorithms)
    
    
    # register the evaluation function
    toolbox.register("evaluate", evaluateFunction,ds) # register a method to evaluate the fitness of an individual
    return toolbox

60

In [ ]:
import copy
from select import select
import time
from turtle import update

def run_GA_framework(ds:DatasetPart2, max_gen= CONSTANTS_DICT["MAX_GENERATIONS"] ,randSeed:int=1) -> creator.Individual:
    '''
    Run the genetic algorithm framework
    '''
    # for toolbox
    random.seed(randSeed)
    toolbox = setup_toolbox(ds,randSeed)
    # for record keeping
    logbook = tools.Logbook()    
    # assign the stats for recording the computational time
    stats = tools.Statistics()
    stats.register("mean", np.mean, axis = 0)
    stats.register("std", np.std, axis=0)
  
    # create the initial population
    population = toolbox.PopulationCreator(n=CONSTANTS_DICT["POPULATION_SIZE"])
    
    # # evaluate the fitness of the current population, and assign the fitness to each individual
    # evaluate_fitness_values(population)
    def evaluate_fitness_values(pop) -> None:
        """Update the fitness values of each individual for the given the population"""
        fitnesses = list(map(toolbox.evaluate, pop))
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit
    
    
    best_feasible_individual = None
    computation_time_list = [] 
    # start the evolution
    for gen_counter in range(max_gen):
        # assign the fitness to each individual in the current generation 
        evaluate_fitness_values(population)
        # for recording the time spent on each generation
        genStartTime = time.time()
        time_cost = computation_time_list[-1] - genStartTime if len(computation_time_list) > 0 \
                                                             else 0.0
        computation_time_list.append(time_cost)
        # for visualizing whether the algorithm is still running
        best_feasible_individual = tools.selBest(population, k=5)[0]
        best_fitness = best_feasible_individual.fitness.values[0]
        # record the statistics of the current generation
        record = stats.compile(computation_time_list) 
        logbook.record(gen=gen_counter,best_fitness=best_fitness, **record)
        
        # apply elitism to obtain the best individuals in the current generation
        offspring = toolbox.elitism(population)

        # repeat until the offspring has the same size as the population
        while len(offspring) < CONSTANTS_DICT["POPULATION_SIZE"]:
            # apply selection
            parent1,parent2 = toolbox.select(population)

            # apply crossover
            c1,c2 = toolbox.mate(copy.deepcopy(parent1),copy.deepcopy(parent2))
            
            # apply mutation to the children
            for child in [c1,c2]:
                if random.random() < CONSTANTS_DICT["MUTATION_RATE"]:
                    toolbox.mutate(child)
                    del child.fitness.values
                # append the children to the offspring
                offspring.append(child)
            # TODO: apply local search to the children
            # annoying, time consuming, not implemented yet although Yi's github got tutorial

        # replace the current population with the offspring new gwneration
        population[:] = offspring
        
           
    return best_feasible_individual, logbook, stats


In [ ]:
def run_5_times_with_different_seeds(ds:DatasetPart2, title:str, max_gen=CONSTANTS_DICT["MAX_GENERATIONS"],randSeed = [i+1 for i in range(5)],run_times=5):
    '''
    Run the GA framework 5 times with different seeds
    '''
    best_5_avg_fitness_list, best_individual_list = [],[]
    gen = None
    for i in range(run_times):
        best_feasible_individual,logbook,stats = run_GA_framework(ds,max_gen,randSeed[i])
        best_individual_list.append(best_feasible_individual)
        print('-'*80)
        print('-'*80)
        print("Running GA with seed: ", randSeed[i])
        print('Best fitness: ', best_feasible_individual.fitness.values[0])
        print('Best value: ', get_weight_value_attrs_for_given_individual(ds,best_feasible_individual)["best_ind_value"])
        print('Best weight: ', get_weight_value_attrs_for_given_individual(ds,best_feasible_individual)["best_ind_weight"])
        print('Best chromosome: ', get_weight_value_attrs_for_given_individual(ds,best_feasible_individual)["best_ind_chromosome"])
        print("FOllowing are the statistics for each generation with the seed: ", randSeed[i])
        print('-'*80)
        logbook.header = "gen", "avg", "std", "min", "max", "best_ind_value", "best_ind_weight", "best_5_avg_fitness","best_ind_chromosome"
        print(logbook)
        gen = logbook.select("gen")
        # best_feasible_individuals.append(best_feasible_individual)
        best_5_avg_fitness_list.append( logbook.select("best_5_avg_fitness"))
        print('-'*80)
        print('-'*80)
        print('-'*80)
    
    for i in range(run_times):
        plt.plot(gen, best_5_avg_fitness_list[i], label=f"Seed: {str(randSeed[i])}")
        plt.legend(loc="lower right")
        # print(best_5_avg_fitness_list[i])
    plt.xlabel("Generation")
    plt.ylabel("Average Fitness of 5 best solutions")
    plt.title(f"dataset: {title}\n 5 Curves for 5 runs ")
    # for addressing the issue of log scale, which is
    # very large and a very small fitness values in a plot
    needSymlog = lambda y_values : min(y_values) < -1e-3
    if needSymlog(best_5_avg_fitness_list[0]):
        plt.yscale('symlog') 
    plt.show()
    
    # OR just one curve??
    avg_5runs = []
    for g in range(len(gen)):
        # for i in range(5):
        #     avg_5runs.append(best_5_avg_fitness_list[i][g])
        avg_5runs.append( np.mean([ best_5_avg_fitness_list[i][g] for i in range(run_times) ]) )
        
    # fig, ax1 = plt.subplots()
    plt.plot(gen, avg_5runs, label="avg fitness for 5 runs and 5 best ind ")
        # plt.legend(loc="lower right")
    plt.xlabel("Generation")
    plt.ylabel("Average Fitness of 5 best solutions from 5 runs")
    plt.title(f"dataset: {title}\n Curve for average fitness of 5 best solutions from 5 runs")
    # for addressing the issue of log scale, which is
    # very large and a very small fitness values in a plot
    if needSymlog(best_5_avg_fitness_list[0]):
        plt.yscale('symlog') 
    plt.show()
    
    
    # print out best individual from each run
    for i in range(len(best_individual_list)):
        print(f"\nBest individual for seed {str(randSeed[i])}:\
              \n\t {best_individual_list[i]}\
              \n\t Fitness: {best_individual_list[i].fitness.values[0]}\
              \n\t Value: {get_weight_value_attrs_for_given_individual(ds,best_individual_list[i])['best_ind_value']}\
              \n\t Weight: {get_weight_value_attrs_for_given_individual(ds,best_individual_list[i])['best_ind_weight']}")